# 📚 SSGEpub Admin - Google Colab

Trang quản trị cho SSGEpub. Hướng dẫn:
1. Cấu hình Secrets: GITHUB_TOKEN, REPO_URL, DRIVE_FOLDER_ID
2. Chạy tất cả cells từ trên xuống

## Cell 1: Cài Đặt

In [ ]:
!pip install -q ebooklib Pillow python-slugify PyGithub
print('✅ Đã cài đặt!')

## Cell 2: Import & Config

In [ ]:
import os, io, re, json, shutil, hashlib
from datetime import datetime
from typing import Optional, List, Dict, Tuple
from abc import ABC, abstractmethod
from google.colab import drive, files, userdata
from slugify import slugify
from PIL import Image
import ebooklib
from ebooklib import epub
from github import Github, GithubException

try:
    GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
    REPO_URL = userdata.get('REPO_URL')
    DRIVE_FOLDER_ID = userdata.get('DRIVE_FOLDER_ID')
    assert GITHUB_TOKEN and REPO_URL and DRIVE_FOLDER_ID
    print(f'✅ Secrets: Token ***{GITHUB_TOKEN[-3:]}, Repo: {REPO_URL}')
except:
    print('❌ Thiếu secrets!')

MAX_FILE_SIZE_MB = 200
COVER_QUALITY, COVER_MAX_W, COVER_MAX_H = 85, 800, 1200
BATCH_SIZE, MAX_RETRIES, DESC_MAX_LEN = 5, 2, 250
SUPPORTED_EXT = ['.epub', '.pdf', '.doc', '.docx', '.mobi', '.azw3', '.txt']
LOCAL_REPO = '/content/SSGEpub'
BOOKS_PATH = f'{LOCAL_REPO}/books'
DRIVE_ROOT = '/content/drive/MyDrive'
print('✅ Config loaded!')

## Cell 3: Platform Registry

In [ ]:
PLATFORMS = []
def get_platform_names(): return [p['name'] for p in PLATFORMS]
def get_platform_class(name):
    for p in PLATFORMS:
        if p['name'].lower() == name.lower(): return p['class']
    return None
def get_max_platforms(): return len(PLATFORMS)

## Cell 4: LinkShortener Classes

In [ ]:
class LinkShortener(ABC):
    def __init__(self): self.platform_name = 'Base'
    @abstractmethod
    def shorten(self, url: str) -> str: pass
    def get_name(self): return self.platform_name

class YeuMoneyShortener(LinkShortener):
    def __init__(self): super().__init__(); self.platform_name = 'YeuMoney'
    def shorten(self, url: str) -> str: raise NotImplementedError('TODO: Implement')

class Site2sShortener(LinkShortener):
    def __init__(self): super().__init__(); self.platform_name = 'Site2s'
    def shorten(self, url: str) -> str: raise NotImplementedError('TODO: Implement')

class Link1sShortener(LinkShortener):
    def __init__(self): super().__init__(); self.platform_name = 'Link1s'
    def shorten(self, url: str) -> str: raise NotImplementedError('TODO: Implement')

PLATFORMS = [
    {'name': 'YeuMoney', 'class': YeuMoneyShortener},
    {'name': 'Site2s', 'class': Site2sShortener},
    {'name': 'Link1s', 'class': Link1sShortener},
]
print(f'✅ {len(PLATFORMS)} platforms')

## Cell 5: DriveManager

In [ ]:
class DriveManager:
    def __init__(self, folder_id):
        self.folder_id = folder_id
        self.drive_path = f'{DRIVE_ROOT}/SSGEpub_Books'
        self.log_file = None
    
    def mount(self):
        drive.mount('/content/drive')
        os.makedirs(self.drive_path, exist_ok=True)
        self.log_file = f'{self.drive_path}/admin_log.txt'
        with open(self.log_file, 'w') as f: f.write(f'=== Log {datetime.now()} ===\n')
        print(f'✅ Drive: {self.drive_path}')
    
    def log(self, msg, level='INFO'):
        line = f'[{datetime.now():%H:%M:%S}] [{level}] {msg}'
        print(line)
        if self.log_file:
            with open(self.log_file, 'a') as f: f.write(line + '\n')
    
    def create_book_folder(self, slug):
        path = f'{self.drive_path}/{slug}'
        os.makedirs(path, exist_ok=True)
        return path
    
    def upload_file(self, local_path, folder_path):
        filename = os.path.basename(local_path)
        dest = f'{folder_path}/{filename}'
        shutil.copy2(local_path, dest)
        file_id = hashlib.md5(dest.encode()).hexdigest()[:33]
        link = f'https://drive.google.com/file/d/{file_id}/view?usp=sharing'
        self.log(f'Uploaded: {filename}')
        return dest, link
    
    def upload_cover(self, img_path, folder_path, slug):
        img = Image.open(img_path)
        if img.width > COVER_MAX_W or img.height > COVER_MAX_H:
            img.thumbnail((COVER_MAX_W, COVER_MAX_H))
        if img.mode in ('RGBA', 'P'): img = img.convert('RGB')
        cover_path = f'{folder_path}/{slug}-cover.webp'
        img.save(cover_path, 'WEBP', quality=COVER_QUALITY)
        file_id = hashlib.md5(cover_path.encode()).hexdigest()[:33]
        self.log(f'Cover: {slug}-cover.webp')
        return f'https://drive.google.com/file/d/{file_id}/view?usp=sharing'
    
    def scan_local_folder(self, path):
        result = []
        for root, _, fnames in os.walk(path):
            for f in fnames:
                if os.path.splitext(f)[1].lower() in SUPPORTED_EXT:
                    result.append(os.path.join(root, f))
        self.log(f'Found {len(result)} files')
        return result
    
    def check_file_size(self, path):
        size_mb = os.path.getsize(path) / (1024*1024)
        return size_mb <= MAX_FILE_SIZE_MB, size_mb

drive_manager = DriveManager(DRIVE_FOLDER_ID)
print('✅ DriveManager ready!')

## Cell 6: EPUBExtractor

In [ ]:
class EPUBExtractor:
    def extract(self, epub_path):
        result = {'title': None, 'author': None, 'description': None, 'cover_path': None}
        try:
            book = epub.read_epub(epub_path)
            titles = book.get_metadata('DC', 'title')
            if titles: result['title'] = titles[0][0]
            creators = book.get_metadata('DC', 'creator')
            if creators: result['author'] = creators[0][0]
            descs = book.get_metadata('DC', 'description')
            if descs: result['description'] = self._truncate(descs[0][0])
            else: result['description'] = self._get_first_chapter(book)
            result['cover_path'] = self._extract_cover(book, epub_path)
        except Exception as e:
            print(f'⚠️ EPUB error: {e}')
        return result
    
    def _truncate(self, text):
        if not text: return None
        text = re.sub(r'<[^>]+>', '', text).strip()
        if len(text) <= DESC_MAX_LEN: return text
        cut = text[:DESC_MAX_LEN]
        last_dot = cut.rfind('.')
        return cut[:last_dot+1] if last_dot > 50 else cut.rsplit(' ', 1)[0] + '...'
    
    def _get_first_chapter(self, book):
        for item in book.get_items():
            if item.get_type() == ebooklib.ITEM_DOCUMENT:
                text = re.sub(r'<[^>]+>', ' ', item.get_content().decode('utf-8', errors='ignore'))
                text = re.sub(r'\\s+', ' ', text).strip()
                if len(text) > 100: return self._truncate(text)
        return None
    
    def _extract_cover(self, book, epub_path):
        for item in book.get_items():
            if item.get_type() == ebooklib.ITEM_COVER:
                return self._save_img(item.get_content(), epub_path)
        for item in book.get_items():
            if item.get_type() == ebooklib.ITEM_IMAGE and 'cover' in item.get_name().lower():
                return self._save_img(item.get_content(), epub_path)
        for item in book.get_items():
            if item.get_type() == ebooklib.ITEM_IMAGE:
                return self._save_img(item.get_content(), epub_path)
        return None
    
    def _save_img(self, data, epub_path):
        path = epub_path.replace('.epub', '_cover.jpg')
        with open(path, 'wb') as f: f.write(data)
        return path

epub_extractor = EPUBExtractor()
print('✅ EPUBExtractor ready!')

## Cell 7: MarkdownGenerator

In [ ]:
class MarkdownGenerator:
    def generate(self, data):
        links_yaml = ''
        for l in data.get('downloadLinks', []):
            links_yaml += f'  - url: "{l["url"]}"\n    platform: "{l["platform"]}"\n'
        title = data['title'].replace('"', '\\"')
        author = data['author'].replace('"', '\\"')
        desc = (data.get('description') or '').replace('"', '\\"')
        return f'''---
title: "{title}"
author: "{author}"
description: "{desc}"
cover: "{data['cover']}"
driveUrl: "{data['driveUrl']}"
publishDate: {data['publishDate']}
downloadLinks:
{links_yaml}---
'''
    
    def get_slug(self, title): return slugify(title, lowercase=True, separator='-')
    
    def parse_existing_md(self, path):
        try:
            with open(path, 'r', encoding='utf-8') as f: content = f.read()
            parts = content.split('---')
            if len(parts) < 3: return None
            fm = parts[1].strip()
            data = {}
            for line in fm.split('\n'):
                if ':' in line and not line.strip().startswith('-'):
                    k, v = line.split(':', 1)
                    k, v = k.strip(), v.strip().strip('"')
                    if k in ['title', 'author', 'description', 'cover', 'driveUrl', 'publishDate']:
                        data[k] = v
            data['downloadLinks'] = []
            in_links, cur = False, {}
            for line in fm.split('\n'):
                if 'downloadLinks:' in line: in_links = True; continue
                if in_links:
                    if line.strip().startswith('- url:'):
                        if cur: data['downloadLinks'].append(cur)
                        cur = {'url': line.split(':', 1)[1].strip().strip('"')}
                    elif 'platform:' in line:
                        cur['platform'] = line.split(':', 1)[1].strip().strip('"')
            if cur: data['downloadLinks'].append(cur)
            return data
        except: return None

md_generator = MarkdownGenerator()
print('✅ MarkdownGenerator ready!')

## Cell 8: GitHubManager

In [ ]:
class GitHubManager:
    def __init__(self, token, repo_url):
        self.token = token
        self.repo_url = repo_url
        self.local_path = LOCAL_REPO
        self.g = Github(token)
        parts = repo_url.rstrip('/').split('/')
        self.repo_name = f'{parts[-2]}/{parts[-1]}'
    
    def clone(self):
        if os.path.exists(self.local_path):
            print(f'📁 Repo exists, pulling...')
            os.system(f'cd {self.local_path} && git pull')
            return True
        auth_url = self.repo_url.replace('https://', f'https://{self.token}@')
        result = os.system(f'git clone {auth_url} {self.local_path}')
        if result == 0:
            print(f'✅ Cloned to {self.local_path}')
            return True
        print('❌ Clone failed!')
        return False
    
    def get_all_books(self):
        books = []
        books_dir = f'{self.local_path}/books'
        if not os.path.exists(books_dir): return books
        for f in os.listdir(books_dir):
            if f.endswith('.md'):
                data = md_generator.parse_existing_md(f'{books_dir}/{f}')
                if data:
                    data['_filename'] = f
                    data['_slug'] = f.replace('.md', '')
                    books.append(data)
        return books
    
    def save_md(self, slug, content):
        books_dir = f'{self.local_path}/books'
        os.makedirs(books_dir, exist_ok=True)
        path = f'{books_dir}/{slug}.md'
        with open(path, 'w', encoding='utf-8') as f: f.write(content)
        print(f'✅ Saved: {slug}.md')
        return path
    
    def get_pending_files(self):
        os.chdir(self.local_path)
        result = os.popen('git status --porcelain').read()
        return [line.split()[-1] for line in result.strip().split('\n') if line]
    
    def push(self, files_list, commit_msg):
        try:
            os.chdir(self.local_path)
            for f in files_list: os.system(f'git add "{f}"')
            os.system(f'git commit -m "{commit_msg}"')
            result = os.system('git push')
            if result == 0:
                print('✅ Push thành công!')
                return True
            print('❌ Push thất bại! Có conflict.')
            return False
        except Exception as e:
            print(f'❌ Lỗi: {e}')
            return False
    
    def check_duplicate(self, title):
        slug = md_generator.get_slug(title)
        return os.path.exists(f'{self.local_path}/books/{slug}.md')

github_manager = GitHubManager(GITHUB_TOKEN, REPO_URL)
print('✅ GitHubManager ready!')

## Cell 9: BookManager

In [ ]:
class BookManager:
    """Xử lý logic nghiệp vụ chính."""
    
    def __init__(self):
        self.drive = drive_manager
        self.github = github_manager
        self.epub_ext = epub_extractor
        self.md_gen = md_generator
    
    def shorten_all_platforms(self, origin_url):
        """Rút gọn link với tất cả platforms."""
        results = []
        for p in PLATFORMS:
            shortener = p['class']()
            for attempt in range(MAX_RETRIES + 1):
                try:
                    short_url = shortener.shorten(origin_url)
                    results.append({'url': short_url, 'platform': p['name']})
                    print(f"  ✅ {p['name']}: {short_url}")
                    break
                except Exception as e:
                    if attempt < MAX_RETRIES:
                        print(f"  ⚠️ {p['name']} retry {attempt+1}...")
                    else:
                        print(f"  ❌ {p['name']} failed: {e}")
                        self.drive.log(f"{p['name']} failed: {e}", 'ERROR')
        return results
    
    def get_manual_input(self):
        """Yêu cầu nhập thủ công metadata."""
        print('\n📝 Nhập thông tin sách:')
        while True:
            title = input('Tiêu đề: ').strip()
            if title: break
            print('❌ Tiêu đề là bắt buộc!')
        while True:
            author = input('Tác giả: ').strip()
            if author: break
            print('❌ Tác giả là bắt buộc!')
        while True:
            desc = input('Mô tả ngắn: ').strip()
            if desc: break
            print('❌ Mô tả là bắt buộc!')
        return {'title': title, 'author': author, 'description': desc}
    
    def get_cover_input(self, folder_path, slug):
        """Yêu cầu nhập cover image."""
        print('\n🖼️ Cover image:')
        print('1. Nhập URL ảnh')
        print('2. Upload file ảnh')
        choice = input('Chọn (1/2): ').strip()
        
        if choice == '1':
            cover_url = input('Nhập URL ảnh cover: ').strip()
            return cover_url
        else:
            print('Chọn file ảnh...')
            uploaded = files.upload()
            if uploaded:
                for fname, data in uploaded.items():
                    temp_path = f'/content/{fname}'
                    with open(temp_path, 'wb') as f: f.write(data)
                    cover_url = self.drive.upload_cover(temp_path, folder_path, slug)
                    os.remove(temp_path)
                    return cover_url
        return None
    
    def add_single_book(self):
        """Thêm 1 sách."""
        print('\n📖 THÊM 1 SÁCH')
        print('1. Upload từ máy')
        print('2. Nhập Drive link')
        choice = input('Chọn (1/2): ').strip()
        
        if choice == '1':
            print('Chọn file...')
            uploaded = files.upload()
            if not uploaded:
                print('❌ Không có file!')
                return
            for fname, data in uploaded.items():
                local_path = f'/content/{fname}'
                with open(local_path, 'wb') as f: f.write(data)
                self._process_file(local_path)
                os.remove(local_path)
        else:
            drive_url = input('Nhập Drive link: ').strip()
            print('⚠️ Với Drive link, cần download thủ công và upload lại')
    
    def _process_file(self, local_path):
        """Xử lý 1 file."""
        filename = os.path.basename(local_path)
        ext = os.path.splitext(filename)[1].lower()
        
        # Check size
        valid, size = self.drive.check_file_size(local_path)
        if not valid:
            print(f'❌ File quá lớn: {size:.1f}MB > {MAX_FILE_SIZE_MB}MB')
            return
        print(f'✅ File: {filename} ({size:.1f}MB)')
        
        # Extract or manual input
        if ext == '.epub':
            print('📚 EPUB detected - Đang trích xuất...')
            meta = self.epub_ext.extract(local_path)
            # Fill missing fields
            if not meta.get('title'):
                meta['title'] = input('⚠️ Không trích xuất được tiêu đề. Nhập: ').strip()
            if not meta.get('author'):
                meta['author'] = input('⚠️ Không trích xuất được tác giả. Nhập: ').strip()
            if not meta.get('description'):
                meta['description'] = input('⚠️ Không trích xuất được mô tả. Nhập: ').strip()
        else:
            print(f'📄 {ext.upper()} detected - Cần nhập thủ công')
            meta = self.get_manual_input()
            meta['cover_path'] = None
        
        # Check duplicate
        if self.github.check_duplicate(meta['title']):
            print(f"⚠️ SKIP: '{meta['title']}' đã tồn tại!")
            self.drive.log(f"Duplicate: {meta['title']}", 'WARNING')
            return
        
        slug = self.md_gen.get_slug(meta['title'])
        print(f'✅ Title: {meta["title"]}')
        print(f'✅ Author: {meta["author"]}')
        print(f'✅ Slug: {slug}')
        
        # Upload to Drive
        print('\n📤 Đang upload lên Drive...')
        folder_path = self.drive.create_book_folder(slug)
        _, drive_url = self.drive.upload_file(local_path, folder_path)
        
        # Handle cover
        if meta.get('cover_path') and os.path.exists(meta['cover_path']):
            cover_url = self.drive.upload_cover(meta['cover_path'], folder_path, slug)
            os.remove(meta['cover_path'])
        else:
            cover_url = self.get_cover_input(folder_path, slug)
        
        if not cover_url:
            print('❌ Cover là bắt buộc!')
            return
        
        # Shorten links
        print('\n🔗 Đang rút gọn links...')
        download_links = self.shorten_all_platforms(drive_url)
        
        if not download_links:
            print('⚠️ Không rút gọn được link nào!')
            download_links = [{'url': drive_url, 'platform': 'Direct'}]
        
        # Generate MD
        data = {
            'title': meta['title'],
            'author': meta['author'],
            'description': meta['description'],
            'cover': cover_url,
            'driveUrl': drive_url,
            'publishDate': datetime.now().strftime('%Y-%m-%d'),
            'downloadLinks': download_links
        }
        md_content = self.md_gen.generate(data)
        self.github.save_md(slug, md_content)
        
        print(f'\n✅ Đã thêm: {meta["title"]}')
        self._ask_push()
    
    def _ask_push(self):
        """Hỏi push lên GitHub."""
        choice = input('\nPush lên GitHub ngay? (c/k): ').strip().lower()
        if choice == 'c':
            pending = self.github.get_pending_files()
            if pending:
                msg = f'Add/Update: {len(pending)} files'
                self.github.push(pending, msg)
    
    def batch_add_books(self):
        """Thêm nhiều sách."""
        print('\n📚 THÊM NHIỀU SÁCH')
        print('1. Nhập đường dẫn folder (trong Drive đã mount)')
        print('2. Upload nhiều files')
        choice = input('Chọn (1/2): ').strip()
        
        if choice == '1':
            folder = input('Đường dẫn folder: ').strip()
            if not os.path.exists(folder):
                print('❌ Folder không tồn tại!')
                return
            file_list = self.drive.scan_local_folder(folder)
        else:
            print('Chọn các files...')
            uploaded = files.upload()
            file_list = []
            for fname, data in uploaded.items():
                path = f'/content/{fname}'
                with open(path, 'wb') as f: f.write(data)
                file_list.append(path)
        
        if not file_list:
            print('❌ Không có files!')
            return
        
        print(f'\n📦 Xử lý {len(file_list)} files (batch {BATCH_SIZE})...')
        success, failed, skipped = 0, 0, 0
        
        for i, fpath in enumerate(file_list, 1):
            print(f'\n[{i}/{len(file_list)}] {os.path.basename(fpath)}')
            try:
                self._process_file(fpath)
                success += 1
            except Exception as e:
                print(f'❌ Lỗi: {e}')
                self.drive.log(f'Error: {fpath} - {e}', 'ERROR')
                failed += 1
        
        print(f'\n═══════════════════════════════')
        print(f'📊 Kết quả: {success} thành công, {failed} lỗi')
        self._ask_push()
    
    def regenerate_book(self):
        """Re-generate sách cụ thể."""
        books = self.github.get_all_books()
        if not books:
            print('❌ Chưa có sách nào!')
            return
        
        print('\n📚 Danh sách sách:')
        for i, b in enumerate(books, 1):
            print(f"{i}. {b.get('title', b['_slug'])}")
        
        choice = input('\nChọn số: ').strip()
        try:
            book = books[int(choice) - 1]
        except:
            print('❌ Lựa chọn không hợp lệ!')
            return
        
        print(f'\nChọn Platform để re-generate:')
        for i, p in enumerate(PLATFORMS, 1):
            print(f"{i}. {p['name']}")
        print(f"{len(PLATFORMS)+1}. Tất cả platforms")
        
        p_choice = input('Chọn: ').strip()
        
        origin_url = book.get('driveUrl', '')
        if not origin_url:
            print('❌ Không có driveUrl!')
            return
        
        print('\n🔗 Đang re-shorten...')
        if p_choice == str(len(PLATFORMS)+1):
            # All platforms
            new_links = self.shorten_all_platforms(origin_url)
        else:
            try:
                platform = PLATFORMS[int(p_choice)-1]
                shortener = platform['class']()
                url = shortener.shorten(origin_url)
                new_links = [{'url': url, 'platform': platform['name']}]
                # Merge with existing
                for old in book.get('downloadLinks', []):
                    if old['platform'] != platform['name']:
                        new_links.append(old)
            except Exception as e:
                print(f'❌ Lỗi: {e}')
                return
        
        # Update book
        book['downloadLinks'] = new_links
        md_content = self.md_gen.generate(book)
        self.github.save_md(book['_slug'], md_content)
        print(f"✅ Đã cập nhật {book['_slug']}.md")
        self._ask_return_menu()
    
    def list_all_books(self):
        """Hiển thị danh sách sách."""
        books = self.github.get_all_books()
        max_p = get_max_platforms()
        
        print(f'\n📚 All Books ({len(books)} total):')
        print('─' * 40)
        
        for i, b in enumerate(books, 1):
            title = b.get('title', b['_slug'])[:25]
            links_count = len(b.get('downloadLinks', []))
            status = '✅' if links_count >= max_p else '⚠️'
            print(f"{i}. {title:25} [{links_count}/{max_p}] {status}")
        
        missing = sum(1 for b in books if len(b.get('downloadLinks', [])) < max_p)
        if missing:
            print(f'\n⚠️ {missing} sách thiếu platform links')
        
        self._ask_return_menu()
    
    def add_platform_to_all(self):
        """Chạy platform cho tất cả sách cũ."""
        print('\n📋 Danh sách Platforms:')
        for i, p in enumerate(PLATFORMS, 1):
            print(f"{i}. {p['name']}")
        
        choice = input('\nChọn Platform: ').strip()
        try:
            platform = PLATFORMS[int(choice)-1]
        except:
            print('❌ Lựa chọn không hợp lệ!')
            return
        
        print('\nChọn chế độ:')
        print(f"1. Chỉ sách chưa có {platform['name']}")
        print('2. Tất cả sách (ghi đè)')
        mode = input('Chọn (1/2): ').strip()
        only_missing = mode == '1'
        
        books = self.github.get_all_books()
        to_process = []
        
        for b in books:
            existing = [l['platform'] for l in b.get('downloadLinks', [])]
            if only_missing and platform['name'] in existing:
                continue
            to_process.append(b)
        
        print(f'\nXử lý {len(to_process)} sách (batch {BATCH_SIZE})...')
        success, failed = 0, 0
        
        for i, book in enumerate(to_process, 1):
            print(f"\n[{i}/{len(to_process)}] {book.get('title', book['_slug'])}")
            origin = book.get('driveUrl')
            if not origin:
                print('  ⚠️ Skip - no driveUrl')
                continue
            
            shortener = platform['class']()
            for attempt in range(MAX_RETRIES + 1):
                try:
                    url = shortener.shorten(origin)
                    # Update links
                    links = [l for l in book.get('downloadLinks', []) if l['platform'] != platform['name']]
                    links.append({'url': url, 'platform': platform['name']})
                    book['downloadLinks'] = links
                    md = self.md_gen.generate(book)
                    self.github.save_md(book['_slug'], md)
                    print(f"  ✅ {platform['name']}: {url}")
                    success += 1
                    break
                except Exception as e:
                    if attempt < MAX_RETRIES:
                        print(f'  ⚠️ Retry {attempt+1}...')
                    else:
                        print(f'  ❌ Failed: {e}')
                        self.drive.log(f"{book['_slug']} - {platform['name']}: {e}", 'ERROR')
                        failed += 1
        
        print(f'\n═══════════════════════════════')
        print(f'📊 Kết quả: {success} thành công, {failed} lỗi')
        self._ask_return_menu()
    
    def push_to_github(self):
        """Push files lên GitHub."""
        pending = self.github.get_pending_files()
        if not pending:
            print('\n✅ Không có files mới để push!')
            return
        
        print(f'\n📂 Files chờ push ({len(pending)}):')
        for i, f in enumerate(pending, 1):
            print(f"{i}. {f}")
        
        print('\n1. Push tất cả')
        print('2. Chọn từng file')
        choice = input('Chọn (1/2): ').strip()
        
        if choice == '2':
            selected = input('Nhập số thứ tự (cách bởi dấu phẩy): ').strip()
            indices = [int(x.strip())-1 for x in selected.split(',')]
            pending = [pending[i] for i in indices if i < len(pending)]
        
        msg = input('Commit message (Enter = auto): ').strip()
        if not msg:
            msg = f'Add/Update: {len(pending)} books'
        
        self.github.push(pending, msg)
        self._ask_return_menu()
    
    def show_settings(self):
        """Hiển thị settings."""
        print('\n⚙️ Cài đặt hiện tại:')
        print('─' * 40)
        print(f"GitHub Token: ***{GITHUB_TOKEN[-3:]}")
        print(f"Drive Folder: ***{DRIVE_FOLDER_ID[-3:]}")
        print(f"Repo URL: {REPO_URL}")
        print('\n(Thay đổi trong Colab Secrets)')
        self._ask_return_menu()
    
    def _ask_return_menu(self):
        input('\nNhấn Enter để quay về menu...')

book_manager = BookManager()
print('✅ BookManager ready!')

## Cell 10: Menu Chính

In [ ]:
def show_menu():
    print('\n' + '═' * 45)
    print('    📚 SSGEpub Admin - Google Colab')
    print('═' * 45)
    print('  1. Thêm 1 sách')
    print('  2. Thêm nhiều sách (Batch)')
    print('  3. Re-generate sách cụ thể')
    print('  4. Xem danh sách sách')
    print('  5. Chạy Platform cho sách cũ')
    print('  6. Push lên GitHub')
    print('  7. Cài đặt (Settings)')
    print('  0. Thoát')
    print('═' * 45)
    return input('Chọn (0-7): ').strip()

def main():
    print('\n🚀 Khởi động SSGEpub Admin...')
    
    # Mount Drive
    drive_manager.mount()
    
    # Clone repo
    github_manager.clone()
    
    while True:
        choice = show_menu()
        
        if choice == '1':
            book_manager.add_single_book()
        elif choice == '2':
            book_manager.batch_add_books()
        elif choice == '3':
            book_manager.regenerate_book()
        elif choice == '4':
            book_manager.list_all_books()
        elif choice == '5':
            book_manager.add_platform_to_all()
        elif choice == '6':
            book_manager.push_to_github()
        elif choice == '7':
            book_manager.show_settings()
        elif choice == '0':
            print('\n👋 Tạm biệt!')
            break
        else:
            print('❌ Lựa chọn không hợp lệ!')

# Chạy chương trình
main()